# 🚀 Colab Direct Download to Drive

Download files directly from any public URL to your Google Drive, with live progress bars, upload tracking, and automatic duplicate filename handling.

---

## 📋 What does this notebook do?

- Downloads files from direct download links (HTTP/HTTPS)  
- Uploads each file into your Google Drive (`MyDrive/ColabDownloads` by default)  
- Shows progress bars for both download and upload steps (speed, ETA, size)

---

## ✅ How to use

1. **Run all cells in order.**
2. When prompted, **paste the direct download URLs** you want (one per line).
3. Type `exit` and press Enter when finished.
4. Your files will be saved in your Google Drive under  
   `My Drive/ColabDownloads`

---

**Requirements**

- Run this notebook inside [Google Colab](https://colab.research.google.com).
- You'll be prompted to mount your Google Drive for saving files.


In [ ]:
# Install required packages
!pip -q install requests tqdm

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Imports
import os, sys, time, uuid, shutil
from pathlib import Path
import requests
from tqdm import tqdm

# Helper: create a unique filename if the target exists
def _unique_name(path: Path) -> Path:
    if not path.exists():
        return path
    stem, suffix = path.stem, path.suffix
    while True:
        candidate = path.with_name(f"{stem}_{uuid.uuid4().hex[:6]}{suffix}")
        if not candidate.exists():
            return candidate

# Download with progress bar
def download_to_tmp(url: str, chunk_size: int = 1024 * 1024) -> Path:
    response = requests.get(url, stream=True, timeout=10)
    response.raise_for_status()

    total = int(response.headers.get("Content-Length", 0))
    local_tmp = Path("/tmp") / os.path.basename(url.split("?")[0])
    local_tmp = _unique_name(local_tmp)

    with open(local_tmp, "wb") as f, tqdm(
        total=total,
        unit="B",
        unit_scale=True,
        unit_divisor=1024,
        desc=f"↓ {local_tmp.name}",
        ascii=True,
        file=sys.stdout,
    ) as bar:
        start = time.time()
        for chunk in response.iter_content(chunk_size=chunk_size):
            if chunk:
                f.write(chunk)
                bar.update(len(chunk))
        elapsed = time.time() - start
    print(f"Downloaded to {local_tmp} in {elapsed:.1f}s")
    return local_tmp

# Upload to Drive with progress bar
def upload_to_drive(local_file: Path, drive_dir: Path, chunk_size: int = 1024 * 1024):
    drive_dir.mkdir(parents=True, exist_ok=True)
    dest_file = _unique_name(drive_dir / local_file.name)

    total = local_file.stat().st_size
    with open(local_file, "rb") as src, open(dest_file, "wb") as dst, tqdm(
        total=total,
        unit="B",
        unit_scale=True,
        unit_divisor=1024,
        desc=f"↑ {dest_file.name}",
        ascii=True,
        file=sys.stdout,
    ) as bar:
        start = time.time()
        while True:
            buf = src.read(chunk_size)
            if not buf:
                break
            dst.write(buf)
            bar.update(len(buf))
        elapsed = time.time() - start
    print(f"Uploaded to {dest_file} in {elapsed:.1f}s\n")

In [ ]:
# Main
def main():
    default_drive_path = Path("/content/drive/MyDrive/ColabDownloads")
    print("Type direct-download URLs (one per line). Enter 'exit' to run.\n")
    urls = []
    while True:
        url = input("URL ➜ ").strip()
        if url.lower() == "exit":
            break
        if url:
            urls.append(url)

    if not urls:
        print("No URLs provided — exiting."); return

    print(f"\nFiles will be saved under: {default_drive_path}\n")
    for url in urls:
        try:
            tmp_file = download_to_tmp(url)
            upload_to_drive(tmp_file, default_drive_path)
            tmp_file.unlink(missing_ok=True)  # clean up
        except Exception as e:
            print(f"❌  Failed for {url}: {e}\n")

if __name__ == "__main__":
    main()